## Fuse boundaries of component segmentations and apply 3D watershed

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt
from skimage import morphology, measure, io
from PIL import Image
import glob

import os
from skimage.segmentation import find_boundaries
import shutil
import tifffile

s1 = io.imread("path to segmentation result by P1 in .tif")
s2 = io.imread("path to segmentation result by P2 in .tif")

bnd_im1=find_boundaries(s1, mode='inner').astype(np.uint8)
bnd_im2=find_boundaries(s2, mode='inner').astype(np.uint8)

bnd_res= np.array(bnd_im1)+ np.array(bnd_im2)
bnd_res=np.array(bnd_res, dtype=np.uint8)

io.imsave('/fused_boundary_image.tif', bnd_res*255)

## A 3D watershed algorithm is to be applied to this fused boundary image. Please use 3D_watershed.yaml environment file to install dependencies for this method

In [ ]:
import os

from time import time as current_time

import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt
from skimage import morphology, measure, io
from timagetk.components import SpatialImage
from timagetk.io import imread, imsave
from timagetk.plugins import h_transform, region_labeling, segmentation

from timagetk.visu.util import glasbey

In [ ]:
def labelled_image_projection(seg_img, axis=2, direction=1, background_label=1, return_coords=False):
    """Compute the 2D projection of a labelled image along the specified  axis

    Parameters
    ----------
    seg_img : LabelledImage
        Labelled image to project in 2D
    axis : int
        The axis along which to project the labelled image
    direction : int
        On which side of the image to project (-1 or 1)
    background_label : uint
        Ignored value for projection
    return_coords : bool
        Whether to return the z-coordinates used for the projection

    Returns
    -------
    np.ndarray
        2D Projected labelled image

    """
    xxx, yyy, zzz = np.mgrid[0:seg_img.shape[0], 0:seg_img.shape[1], 0:seg_img.shape[2]].astype(float)

    if axis == 0:
        y = np.arange(seg_img.shape[1])
        z = np.arange(seg_img.shape[2])
        yy, zz = map(np.transpose, np.meshgrid(y, z))
        proj = xxx * (seg_img.get_array() != background_label)
    elif axis == 1:
        x = np.arange(seg_img.shape[0])
        z = np.arange(seg_img.shape[2])
        xx, zz = map(np.transpose, np.meshgrid(x, z))
        proj = yyy * (seg_img.get_array() != background_label)
    elif axis == 2:
        x = np.arange(seg_img.shape[0])
        y = np.arange(seg_img.shape[1])
        xx, yy = map(np.transpose, np.meshgrid(x, y))
        proj = zzz * (seg_img.get_array() != background_label)

    proj[proj == 0] = np.nan
    if direction == 1:
        proj = np.nanmax(proj, axis=axis)
        proj[np.isnan(proj)] = seg_img.shape[axis] - 1
    elif direction == -1:
        proj = np.nanmin(proj, axis=axis)
        proj[np.isnan(proj)] = 0

    if axis == 0:
        xx = proj
    elif axis == 1:
        yy = proj
    elif axis == 2:
        zz = proj

  
    coords = tuple(np.transpose(np.concatenate(np.transpose([xx, yy, zz], (1, 2, 0)).astype(int))))
    projected_img = np.transpose(seg_img.get_array()[coords].reshape(xx.shape))

    if not return_coords:
        return projected_img
    else:
        return projected_img, coords

In [ ]:
dirname = 'path to fused boundary image in .tif'
fname = "fused_boundary_image.tif"

In [ ]:
h_min =0
gaussian_sigma = 0.7
segmentation_gaussian_sigma = 0.5

start_time = current_time()
print("--> Loading membrane images...")
membrane_images = {}
for image_id in range(1):
    image_filename = dirname + "/" + fname
    if os.path.exists(image_filename):
        print("  --> Loading membrane image "+str(image_id).zfill(2))
        membrane_images[image_id] = imread(image_filename) 
print("<-- Loading "+str(len(membrane_images))+" membrane images ["+str(current_time()-start_time)+"s]")

In [ ]:
print("--> Segmenting membrane images...")
segmented_images = {}
for image_id in membrane_images.keys():
    start_time = current_time()
    print("  --> Segmenting membrane image "+str(image_id).zfill(2))
    img = membrane_images[image_id]

    voxelsize = np.array(img.voxelsize)

    step_start_time = current_time()
    smooth_image = nd.gaussian_filter(img, sigma=gaussian_sigma / voxelsize).astype(img.dtype)
    smooth_img = SpatialImage(smooth_image, voxelsize=voxelsize)
    print("    --> Smoothing image for seed detection ["+str(current_time()-step_start_time)+"s]")

    step_start_time = current_time()
    ext_img = h_transform(smooth_img, h=h_min, method='min')
    print("    --> Computing H-transform ["+str(current_time()-step_start_time)+"s]")

    step_start_time = current_time()
    seed_img = region_labeling(ext_img, low_threshold=1, high_threshold=h_min, method='connected_components')
    print("    --> Extracting seed image ["+str(current_time()-step_start_time)+"s]")

    step_start_time = current_time()
    seg_smooth_image = nd.gaussian_filter(img, sigma=segmentation_gaussian_sigma / voxelsize).astype(img.dtype)
    seg_smooth_img = SpatialImage(seg_smooth_image, voxelsize=voxelsize)
    print("    --> Smoothing image for segmentation ["+str(current_time()-step_start_time)+"s]")

    step_start_time = current_time()
    seg_img = segmentation(seg_smooth_img, seed_img, control='first', method='seeded_watershed')
    segmented_images[image_id] = seg_img
    print("    --> Performing watershed segmentation ["+str(current_time()-step_start_time)+"s]")
    print("  <-- Segmenting membrane image "+str(image_id).zfill(2)+" ["+str(current_time()-start_time)+"s]")

    start_time = current_time()
    print("  --> Saving segmented image "+str(image_id).zfill(2))
    imsave(dirname+"/"+fname+ "_3D_segmentation.tif",seg_img)
    print("  <-- Saving segmented image "+str(image_id).zfill(2)+" ["+str(current_time()-start_time)+"s]")

In [ ]:
print("--> Displaying segmentation results...")
figure = plt.figure(0)
figure.clf()
for i_image, image_id in enumerate(segmented_images.keys()):
    img = membrane_images[image_id]
    seg_img = segmented_images[image_id]

    start_time = current_time()
    print("  --> Projecting segmented image "+str(image_id).zfill(2))
    projected_seg_img, projection_coords = labelled_image_projection(seg_img,direction=-1,return_coords=True)
    print("  <-- Projecting segmented image "+str(image_id).zfill(2)+" ["+str(current_time()-start_time)+"s]")

    start_time = current_time()
    print("  --> Displaying images ")
    figure.add_subplot(len(segmented_images),2,2*i_image+1)

    figure.gca().imshow(img[projection_coords].reshape(projected_seg_img.shape).T,cmap='gray',vmin=0,vmax=255)
    figure.gca().set_title("Membrane Image ",size=20)
    figure.gca().axis('off')

    figure.add_subplot(len(segmented_images),2,2*i_image+2)

    figure.gca().imshow(projected_seg_img%256,cmap='glasbey',vmin=0,vmax=255)
    figure.gca().set_title("Segmented Image ",size=20)
    figure.gca().axis('off')
    print("  <-- Displaying images "+str(image_id).zfill(2)+" ["+str(current_time()-start_time)+"s]")

figure.set_size_inches(20,10*len(segmented_images))
figure.tight_layout()